In [2]:
import pandas as pd
import numpy as np

val_file = "../data/filtered_deconvolved_final_validation_set.csv"
train_file = "../data/final_training_set.csv"

val_df = pd.read_csv(val_file)
train_df = pd.read_csv(train_file)

val_df.head()

,epitope,label,MHC_expanded,peptide_length,key_binder,hla_sequence,predict_on,logit_1,logit_2,final_predictions,Length,predict_on_len
0,LSTVGPRL,1,C0304,8.0,537649.0,MRVMAPRTLILLLSGALALTETWAGSHSMRYFYTAVSRPGRGEPHF...,MRVMAPRTLILLLSGALALTETWAGSHSMRYFYTAVSRPGRGEPHF...,-1.635385,1.839033,0.862834,NaN,NaN
1,AETTTLFQF,1,B4403,9.0,317702.0,MRVTAPRTLLLLLWGAVALTETWAGSHSMRYFYTAMSRPGRGEPRF...,MRVTAPRTLLLLLWGAVALTETWAGSHSMRYFYTAMSRPGRGEPRF...,-3.080864,3.342883,0.965871,NaN,NaN
2,LPMKVRALGL,1,B0702,10.0,518233.0,MLVMAPRTVLLLLSAALALTETWAGSHSMRYFYTSVSRPGRGEPRF...,MLVMAPRTVLLLLSAALALTETWAGSHSMRYFYTSVSRPGRGEPRF...,-2.684186,2.932315,0.949421,NaN,NaN
3,DIQSSGRAK,1,A0301,9.0,12400.0,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,3.406410,-3.716567,0.023740,NaN,NaN
4,TMGHHTVGLK,1,A0301,10.0,308039.0,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,1.666246,-1.937590,0.125913,NaN,NaN


In [3]:
train_df.head()

,Allele,Length,Peptide,label,allele_sequence,predict_on,predict_on_len,training,logit_1,logit_2,isotonic
0,A0101,8,ADMGHLKY,1,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,373,1,-2.578428,2.831741,0.974387
1,A0101,9,GHQVALSSI,0,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,374,1,4.118745,-4.410598,0.000687
2,A0101,11,KQKEVFLPSTP,0,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,376,1,3.885854,-4.178602,0.002166
3,A0101,11,LLSNSSSLWRS,0,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,376,1,3.744931,-4.059163,0.002585
4,A0101,11,AVQGGRLSAIT,0,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,376,1,3.169343,-3.485533,0.011797


In [6]:
[x for x in train_df.Allele.drop_duplicates() if x not in list(val_df.MHC_expanded.drop_duplicates())]

['A0202',
 'A0211',
 'A1102',
 'A2407',
 'A3401',
 'A3402',
 'A3601',
 'A7401',
 'B0704',
 'B3507',
 'B3802',
 'B4006',
 'B5301',
 'B5502',
 'C0302',
 'C0403',
 'C1403']

In [11]:
allele_dict = {}

allele_list = list(set(train_df.Allele).union(set(val_df.MHC_expanded)))
for allele in allele_list:
    if allele in list(val_df.MHC_expanded):
        allele_dict[allele] = list(val_df.loc[val_df.MHC_expanded == allele, 'hla_sequence'])[0]
    else:
        allele_dict[allele] = list(train_df.loc[train_df['Allele'] == allele, 'allele_sequence'])[0]

striated_allele_dict = {}
striated_allele_dict['A'] = {}
striated_allele_dict['B'] = {}
striated_allele_dict['C'] = {}
for key,value in allele_dict.items():
    if key.startswith('A'):
        striated_allele_dict['A'][key] = value
    elif key.startswith('B'):
        striated_allele_dict['B'][key] = value
    elif key.startswith('C'):
        striated_allele_dict['C'][key] = value

striated_allele_dict

{'A': {'A1102': 'MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFYTSVSRPGRGKPRFIAVGYVDDTQFVRFDSDAASQRMEPRAPWIEQEGPEYWDQETRNVKAQSQTDRVDLGTLRGYYNQSEDGSHTIQIMYGCDVGPDGRFLRGYRQDAYDGKDYIALNEDLRSWTAADMAAQITKRKWEAAHAAEQQRAYLEGRCVEWLRRYLENGKETLQRTDPPKTHMTHHPISDHEATLRCWALGFYPAEITLTWQRDGEDQTQDTELVETRPAGDGTFQKWAAVVVPSGEEQRYTCHVQHEGLPKPLTLRWELSSQPTIPIVGIIAGLVLLGAVITGAVVAAVMWRRKSSDRKGGSYTQAASSDSAQGSDVSLTACKV',
  'A0103': 'MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQKMEPRAPWIEQEGPEYWDQETRNMKAHSQTDRANLGTLRGYYNQSEDGSHTIQMMYGCDVGPDGRFLRGYRQDAYDGKDYIALNEDLRSWTAADMAAQITKRKWEAVHAAEQRRVYLEGRCVDGLRRYLENGKETLQRTDPPKTHMTHHPISDHEATLRCWALGFYPAEITLTWQRDGEDQTQDTELVETRPAGDGTFQKWAAVVVPSGEEQRYTCHVQHEGLPKPLTLRWELSSQPTIPIVGIIAGLVLLGAVITGAVVAAVMWRRKSSDRKGGSYTQAASSDSAQGSDVSLTACKV',
  'A6602': 'MAVMAPRTLVLLLSGALALTQTWAGSHSMRYFYTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRMEPRAPWIEQEGPEYWDRNTRNVKAQSQTDRVDLGTLRGYYNQSEAGSHTIQRMYGCDVGPDGRFLRGYQQDAYDGKDYIALNEDLRSWTAADMAAQITQRKWETAHEAEQWRAYLEGECVEWLRRYLENGKETLQRTDAPKTHMTHHAVSDHEA

In [41]:
from tqdm import tqdm
"""
Get a version of striated allele dict where
every allele only contains indices where 
anything is different. Should save some time...
"""
HLA_A_differs = []
#len_a = min([ len(x) for x in striated_allele_dict['A'].values()])
As = striated_allele_dict['A']
for i, aa_a in enumerate(tqdm(As.values())):
    for aa_b in As.values():
        for aa_index, aa_value in enumerate(aa_a):
#            if aa_index >= len_a:
#                break #Just hard-stop at length of A1102
            if aa_index in HLA_A_differs:
                continue #No need to do a comparison if index already captured
            try:
                if aa_value != aa_b[aa_index]:
                    HLA_A_differs.append(aa_index) #add aa index to difference list
            except IndexError:
                continue


HLA_B_differs = []
Bs = striated_allele_dict['B']
for i, aa_a in enumerate(tqdm(Bs.values())):
    for aa_b in Bs.values():
        for aa_index, aa_value in enumerate(aa_a):
#            if aa_index >= len_a:
#                break #Just hard-stop at length of A1102
            if aa_index in HLA_B_differs:
                continue #No need to do a comparison if index already captured
            try:
                if aa_value != aa_b[aa_index]:
                    HLA_B_differs.append(aa_index) #add aa index to difference list
            except IndexError:
                continue



HLA_C_differs = []
Cs = striated_allele_dict['C']
for i, aa_a in enumerate(tqdm(Cs.values())):
    for aa_b in Cs.values():
        for aa_index, aa_value in enumerate(aa_a):
#            if aa_index >= len_a:
#                break #Just hard-stop at length of A1102
            if aa_index in HLA_C_differs:
                continue #No need to do a comparison if index already captured
            try:
                if aa_value != aa_b[aa_index]:
                    HLA_C_differs.append(aa_index) #add aa index to difference list
            except IndexError:
                continue             

cropped_allele_dict = {
    'A': {},
    'B': {},
    'C': {}
}

super_indices = {
    'A': sorted(HLA_A_differs),
    'B': sorted(HLA_B_differs),
    'C': sorted(HLA_C_differs)
}

for supertype, inner_dict in striated_allele_dict.items():
    for allele, allele_sequence in inner_dict.items():
        max_index = len(allele_sequence) - 1
        get_indices = [x for x in super_indices[supertype] if x <= max_index]
        new_aas = "".join([allele_sequence[x] for x in get_indices])
        cropped_allele_dict[supertype][allele] = new_aas

cropped_allele_dict
        

100%|██████████████████████████████████████████████████████████████████████████████████| 27/27 [00:00<00:00, 232.12it/s]


{'A': {'A1102': 'MAVAPRTLLLLLSALALTQTWASHSMRYYTSVRPGRGKPRFIVGYVDDTQFVRFDSDAASQRMEPRAPWIEQEGPEWDQETRNVKAQSQTDRVDLGTLRGYYNQSEDGSHTIQIMYGCDVGPDGRFLRGYRQDAYDGDYILNEDLRSWTAADMAAQITKRKWAAHAAEQRAEREWYPKPIGAAELIPILVITMKTV',
  'A0103': 'MAVAPRTLLLLLSALALTQTWASHSMRYFTSVRPGRGEPRFIVGYVDDTQFVRFDSDAASQKMEPRAPWIEQEGPEWDQETRNMKAHSQTDRANLGTLRGYYNQSEDGSHTIQMMYGCDVGPDGRFLRGYRQDAYDGDYILNEDLRSWTAADMAAQITKRKWAVHAAERRVERDGYPKPIGAAELIPILVITMKTV',
  'A6602': 'MAVAPRTLVLLLSALALTQTWASHSMRYYTSVRPGRGEPRFIVGYVDDTQFVRFDSDAASQRMEPRAPWIEQEGPEWDRNTRNVKAQSQTDRVDLGTLRGYYNQSEAGSHTIQRMYGCDVGPDGRFLRGYQQDAYDGDYILNEDLRSWTAADMAAQITQRKWTAHEAEWRAEEEWYAKAVSASQPIPIFVIAMKSM',
  'A2402': 'MAVAPRTLVLLLSALALTQTWASHSMRYSTSVRPGRGEPRFIVGYVDDTQFVRFDSDAASQRMEPRAPWIEQEGPEWDEETGKVKAHSQTDRENLRIALRYYNQSEAGSHTLQMMFGCDVGSDGRFLRGYHQYAYDGDYILKEDLRSWTAADMAAQITKRKWAAHVAEQRAETDGYPKPIGAAEPVPILVITMNSV',
  'A3002': 'MAVAPRTLLLLLSALALTQTWASHSMRYSTSVRPGSGEPRFIVGYVDDTQFVRFDSDAASQRMEPRAPWIEQERPEWDQETRNVKAHSQTDRENLGTLRGYYNQSEAGSHTIQIMYGCDVGSDGRFLRGYEQHAYDGDY

In [86]:
from scipy import stats
"""
get allele-wise comparisons
basically we need, for A1102: 
How many other alleles have amino acids < 5 apart from it
How many other alleles have amino acids < 3 apart from it
How many other alleles have amino acids < 1 apart from it

Names of alleles with amino acids < 5 apart from it (list)
Names of alleles with amino acids < 3 apart from it (list)
Names of alleles with amino acids < 1 apart from it (list)

Total number of training samples for allele itself
Total number of training samples for alleles < 5 apart from it
Total number of training samples for alleles < 3 apart from it
Total number of training samples for alleles < 1 apart from it
"""

#If a AA sequence doesn't align with mode, just assign 0 
super_modes = {
    'A': stats.mode([len(x) for x in cropped_allele_dict['A'].values()])[0][0],
    'B': stats.mode([len(x) for x in cropped_allele_dict['B'].values()])[0][0],
    'C': stats.mode([len(x) for x in cropped_allele_dict['C'].values()])[0][0]
}



allele_dict_to_df = {}
#Get from self
allele_dict_to_df['allele'] = []
allele_dict_to_df['num_validation_samples'] = []
allele_dict_to_df['num_training_samples'] = []

#Get from everything else
allele_dict_to_df['num_hlas_5_away'] = []
allele_dict_to_df['num_hlas_3_away'] = []
allele_dict_to_df['num_hlas_1_away'] = []
allele_dict_to_df['num_analogs'] = []
allele_dict_to_df['names_hlas_5_away'] = []
allele_dict_to_df['names_hlas_3_away'] = []
allele_dict_to_df['names_hlas_1_away'] = []
allele_dict_to_df['names_analogs'] = []
allele_dict_to_df['num_training_samples_5_away'] = []
allele_dict_to_df['num_training_samples_3_away'] = []
allele_dict_to_df['num_training_samples_1_away'] = []
allele_dict_to_df['num_training_samples_analogs'] = []
allele_dict_to_df['different_length'] = []

entry = 0
for super_type, inner_dict in cropped_allele_dict.items():
    all_alleles = list(inner_dict.keys())
    for idx_a, allele_a in enumerate(all_alleles):
        allele_dict_to_df['allele'].append(allele_a) #allele_name
        allele_dict_to_df['num_validation_samples'].append(len(val_df.loc[val_df.MHC_expanded == allele_a]))
        allele_dict_to_df['num_training_samples'].append(len(train_df.loc[train_df.Allele == allele_a]))

        analogs = []
        five_away = []
        three_away = []
        one_away = []

        seq_a = inner_dict[allele_a]
        if len(seq_a) != super_modes[super_type]:
            allele_dict_to_df['num_hlas_5_away'].append(0)
            allele_dict_to_df['num_hlas_3_away'].append(0)
            allele_dict_to_df['num_hlas_1_away'].append(0)
            allele_dict_to_df['num_analogs'].append(0)
            allele_dict_to_df['names_hlas_5_away'].append([])
            allele_dict_to_df['names_hlas_3_away'].append([])
            allele_dict_to_df['names_hlas_1_away'].append([])
            allele_dict_to_df['names_analogs'].append([])
            allele_dict_to_df['num_training_samples_5_away'].append(0)
            allele_dict_to_df['num_training_samples_3_away'].append(0)
            allele_dict_to_df['num_training_samples_1_away'].append(0)
            allele_dict_to_df['num_training_samples_analogs'].append(0)
            allele_dict_to_df['different_length'].append(True)
            continue
        allele_dict_to_df['different_length'].append(False)
        
        for idx_b, allele_b in enumerate(all_alleles):
            if allele_a == allele_b:
                continue
            seq_b = inner_dict[allele_b]
            names_5 = []
            names_3 = []
            names_1 = []

            #Do the comparison
            difference_val = 0
            for i, aa in enumerate(seq_b):
                if aa != seq_a[i]:
                    difference_val += 1
                    
            if difference_val == 0:
                analogs.append(allele_b)
                five_away.append(allele_b)
                three_away.append(allele_b)
                one_away.append(allele_b)

            elif difference_val == 1:
                one_away.append(allele_b)
                five_away.append(allele_b)
                three_away.append(allele_b)

            elif difference_val < 4:
                three_away.append(allele_b)
                five_away.append(allele_b)

            elif difference_val < 6:
                five_away.append(allele_b)

        #This closes allele_b loop. Now start populating values
        allele_dict_to_df['names_hlas_5_away'].append(five_away)
        allele_dict_to_df['names_hlas_3_away'].append(three_away)
        allele_dict_to_df['names_hlas_1_away'].append(one_away)
        allele_dict_to_df['names_analogs'].append(analogs)

        allele_dict_to_df['num_hlas_5_away'].append(len(five_away))
        allele_dict_to_df['num_hlas_3_away'].append(len(three_away))
        allele_dict_to_df['num_hlas_1_away'].append(len(one_away))
        allele_dict_to_df['num_analogs'].append(len(analogs))
        
        allele_dict_to_df['num_training_samples_5_away'].append(
            sum([len(train_df.query('Allele == @allele_b')) for allele_b in five_away])
        )
        allele_dict_to_df['num_training_samples_3_away'].append(
            sum([len(train_df.query('Allele == @allele_b')) for allele_b in three_away])
        )
        allele_dict_to_df['num_training_samples_1_away'].append(
            sum([len(train_df.query('Allele == @allele_b')) for allele_b in one_away])
        )
        allele_dict_to_df['num_training_samples_analogs'].append(
            sum([len(train_df.query('Allele == @allele_b')) for allele_b in analogs])
        )

        entry += 1


final_allele_df = pd.DataFrame(allele_dict_to_df)
final_allele_df

/tmp/ipykernel_26494/260890451.py:21: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  'A': stats.mode([len(x) for x in cropped_allele_dict['A'].values()])[0][0],
/tmp/ipykernel_26494/260890451.py:22: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  'B': stats.mode([len

,allele,num_validation_samples,num_training_samples,num_hlas_5_away,num_hlas_3_away,num_hlas_1_away,num_analogs,names_hlas_5_away,names_hlas_3_away,names_hlas_1_away,names_analogs,num_training_samples_5_away,num_training_samples_3_away,num_training_samples_1_away,num_training_samples_analogs,different_length
0,A1102,0,11548,1,1,1,0,[A1101],[A1101],[A1101],[],16992,16992,16992,0,False
1,A0103,14,0,2,1,1,0,"[A3601, A0101]",[A0101],[A0101],[],15506,6166,6166,0,False
2,A6602,54,0,4,1,0,0,"[A2601, A2603, A3401, A6601]",[A6601],[],[],24514,8701,0,0,False
3,A2402,17942,10318,3,2,2,0,"[A2407, A2406, A2301]","[A2407, A2406]","[A2407, A2406]",[],17956,6405,6405,0,False
4,A3002,136,11439,3,2,1,0,"[A3004, A3001, A3003]","[A3004, A3003]",[A3003],[],5407,0,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,C1204,6,0,0,0,0,0,[],[],[],[],0,0,0,0,True
156,C1202,524,6442,3,2,1,0,"[C1203, C1604, C1204]","[C1203, C1204]",[C1203],[],10099,10099,10099,0,False
157,C0202,5214,4337,1,1,1,0,[C0210],[C0210],[C0210],[],0,0,0,0,False
158,C0302,0,5612,2,2,0,0,"[C0303, C0304]","[C0303, C0304]",[],[],21035,21035,0,0,False


In [87]:
final_allele_df.to_csv('../data/alleles_pan_numbers.csv', index=False)

In [88]:
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
# precision-recall curve and f1
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc

from sklearn.calibration import CalibratedClassifierCV, CalibrationDisplay
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

def compute_accuracy(y_true, y_pred):
    correct_predictions = 0
    # iterate over each label and check
    for true, predicted in zip(y_true, y_pred):
        if true == predicted:
            correct_predictions += 1
    # compute the accuracy
    accuracy = correct_predictions/len(y_true)
    return accuracy

def get_PRC(df):
    val_x = list(zip(df['logit_1'], df['logit_2']))
    val_y = df['label'].values

    val_yhat = regular_isotonic.predict(val_x)
    val_yhat_prob = [x[1] for x in regular_isotonic.predict_proba(val_x)]


    lr_precision, lr_recall, _ = precision_recall_curve(df['label'], val_yhat_prob)
    lr_f1, lr_auc = f1_score(df['label'], val_yhat), auc(lr_recall, lr_precision)
    # summarize scores
    print('Logistic: f1=%.3f auc=%.3f' % (lr_f1, lr_auc))
    # plot the precision-recall curves
    no_skill = len([x for x in df['label'] if x ==1]) / len(df['label'])
    plt.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill')
    plt.plot(lr_recall, lr_precision, marker='.', label='Predictor')
    # axis labels
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    # show the legend
    plt.legend()
    # show the plot
    plt.show()

train_x = list(zip(train_df['logit_1'], train_df['logit_2']))
train_y = list(train_df['label'])

val_x = list(zip(val_df['logit_1'], val_df['logit_2']))

regular_isotonic = CalibratedClassifierCV(None,  method="isotonic")
regular_isotonic.fit(train_x, train_y)

train_df['isotonic_prob'] = [x[1] for x in regular_isotonic.predict_proba(train_x)]
train_df['isotonic_pred'] = regular_isotonic.predict(train_x)

val_df['isotonic_prob'] = [x[1] for x in regular_isotonic.predict_proba(val_x)]
val_df['isotonic_pred'] = regular_isotonic.predict(val_x)

In [89]:
print(compute_accuracy(val_df['label'], val_df['isotonic_pred']))
print(compute_accuracy(train_df['label'], train_df['isotonic_pred']))

0.90649328884623
0.9801413239197789


In [103]:
final_allele_df['val_accuracy'] = 0.
final_allele_df['training_accuracy'] = 0

final_allele_df['val_f1'] = 0.
final_allele_df['training_f1'] = 0

final_allele_df['val_auc'] = 0.
final_allele_df['training_auc'] = 0


for i, allele in enumerate(tqdm(list(final_allele_df.allele))):
    sub_val = val_df.query('MHC_expanded == @allele').copy()
#    print(allele, len(sub_val))
    try:
        final_allele_df.loc[final_allele_df.allele == allele, 'val_accuracy'] = compute_accuracy(
            sub_val['label'], sub_val['isotonic_pred'])

        lr_precision, lr_recall, _ = precision_recall_curve(sub_val['label'], sub_val['isotonic_prob'])
        lr_f1, lr_auc = f1_score(sub_val['label'], sub_val['isotonic_pred']), auc(lr_recall, lr_precision)

        final_allele_df.loc[final_allele_df.allele == allele, 'val_auc'] = lr_auc
        final_allele_df.loc[final_allele_df.allele == allele, 'val_f1'] = lr_f1
        
    except ZeroDivisionError:
        final_allele_df.loc[final_allele_df.allele == allele, 'val_accuracy'] = np.nan
        final_allele_df.loc[final_allele_df.allele == allele, 'val_f1'] = np.nan
        final_allele_df.loc[final_allele_df.allele == allele, 'val_auc'] = np.nan

    sub_train = train_df.query('Allele == @allele').copy()
#    print(allele, len(sub_val))
    try:
        final_allele_df.loc[final_allele_df.allele == allele, 'training_accuracy'] = compute_accuracy(
        sub_train['label'], sub_train['isotonic_pred'])

        lr_precision, lr_recall, _ = precision_recall_curve(sub_train['label'], sub_train['isotonic_prob'])
        lr_f1, lr_auc = f1_score(sub_train['label'], sub_train['isotonic_pred']), auc(lr_recall, lr_precision)

        final_allele_df.loc[final_allele_df.allele == allele, 'training_auc'] = lr_auc
        final_allele_df.loc[final_allele_df.allele == allele, 'training_f1'] = lr_f1
    
    except ZeroDivisionError:
        final_allele_df.loc[final_allele_df.allele == allele, 'training_accuracy'] = np.nan
        final_allele_df.loc[final_allele_df.allele == allele, 'training_f1'] = np.nan
        final_allele_df.loc[final_allele_df.allele == allele, 'training_auc'] = np.nan

    
final_allele_df

100%|█████████████████████████████████████████████████████████████████████████████████| 160/160 [00:03<00:00, 40.98it/s]


,allele,num_validation_samples,num_training_samples,num_hlas_5_away,num_hlas_3_away,num_hlas_1_away,num_analogs,names_hlas_5_away,names_hlas_3_away,names_hlas_1_away,...,num_training_samples_1_away,num_training_samples_analogs,different_length,accuracy,training_accuracy,val_accuracy,val_f1,training_f1,val_auc,training_auc
0,A1102,0,11548,1,1,1,0,[A1101],[A1101],[A1101],...,16992,0,False,NaN,0.981988,NaN,NaN,0.945805,NaN,0.982141
1,A0103,14,0,2,1,1,0,"[A3601, A0101]",[A0101],[A0101],...,6166,0,False,1.000000,NaN,1.000000,1.000000,NaN,1.000000,NaN
2,A6602,54,0,4,1,0,0,"[A2601, A2603, A3401, A6601]",[A6601],[],...,0,0,False,0.925926,NaN,0.925926,0.923077,NaN,0.988514,NaN
3,A2402,17942,10318,3,2,2,0,"[A2407, A2406, A2301]","[A2407, A2406]","[A2407, A2406]",...,6405,0,False,0.929997,0.990987,0.929997,0.925645,0.972395,0.988669,0.990468
4,A3002,136,11439,3,2,1,0,"[A3004, A3001, A3003]","[A3004, A3003]",[A3003],...,0,0,False,0.941176,0.981817,0.941176,0.937500,0.944798,0.989391,0.976636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,C1204,6,0,0,0,0,0,[],[],[],...,0,0,True,1.000000,NaN,1.000000,1.000000,NaN,1.000000,NaN
156,C1202,524,6442,3,2,1,0,"[C1203, C1604, C1204]","[C1203, C1204]",[C1203],...,10099,0,False,0.816794,0.976560,0.816794,0.782805,0.929735,0.949844,0.972040
157,C0202,5214,4337,1,1,1,0,[C0210],[C0210],[C0210],...,0,0,False,0.862677,0.975790,0.862677,0.844415,0.927134,0.968746,0.969418
158,C0302,0,5612,2,2,0,0,"[C0303, C0304]","[C0303, C0304]",[],...,0,0,False,NaN,0.974519,NaN,NaN,0.924057,NaN,0.971090


In [92]:
final_allele_df.query('num_training_samples_5_away == 0 & num_training_samples == 0')

,allele,num_validation_samples,num_training_samples,num_hlas_5_away,num_hlas_3_away,num_hlas_1_away,num_analogs,names_hlas_5_away,names_hlas_3_away,names_hlas_1_away,names_analogs,num_training_samples_5_away,num_training_samples_3_away,num_training_samples_1_away,num_training_samples_analogs,different_length,accuracy
13,A2413,474,0,0,0,0,0,[],[],[],[],0,0,0,0,True,0.732068
55,B3910,12,0,6,6,1,0,"[B3905, B3906, B3901, B6701, B3924, B3909]","[B3905, B3906, B3901, B6701, B3924, B3909]",[B3901],[],0,0,0,0,False,0.750000
58,B1516,26,0,0,0,0,0,[],[],[],[],0,0,0,0,False,0.923077
74,B4104,100,0,4,2,0,0,"[B4101, B4106, B4102, B4103]","[B4102, B4103]",[],[],0,0,0,0,False,0.810000
79,B4101,1298,0,4,3,2,0,"[B4104, B4106, B4102, B4103]","[B4106, B4102, B4103]","[B4106, B4103]",[],0,0,0,0,False,0.909091
80,B4106,38,0,4,3,1,0,"[B4104, B4101, B4102, B4103]","[B4101, B4102, B4103]",[B4101],[],0,0,0,0,False,0.710526
82,B3906,8402,0,5,5,0,0,"[B3905, B3910, B3901, B3924, B3909]","[B3905, B3910, B3901, B3924, B3909]",[],[],0,0,0,0,False,0.842418
84,B3901,6418,0,7,5,3,0,"[B3905, B3910, B3906, B6701, B1401, B3924, B3909]","[B3905, B3910, B3906, B3924, B3909]","[B3905, B3910, B3909]",[],0,0,0,0,False,0.963228
85,B4406,6,0,0,0,0,0,[],[],[],[],0,0,0,0,False,0.833333
90,B4105,64,0,0,0,0,0,[],[],[],[],0,0,0,0,True,0.609375


In [93]:
final_allele_df.query('num_training_samples_5_away == 0 & num_training_samples != 0')

,allele,num_validation_samples,num_training_samples,num_hlas_5_away,num_hlas_3_away,num_hlas_1_away,num_analogs,names_hlas_5_away,names_hlas_3_away,names_hlas_1_away,names_analogs,num_training_samples_5_away,num_training_samples_3_away,num_training_samples_1_away,num_training_samples_analogs,different_length,accuracy
15,A7401,0,10307,0,0,0,0,[],[],[],[],0,0,0,0,False,NaN
24,A2501,3364,4728,0,0,0,0,[],[],[],[],0,0,0,0,False,0.892985
28,A3201,1012,9870,0,0,0,0,[],[],[],[],0,0,0,0,False,0.916008
31,A2902,14866,4717,1,1,1,0,[A2901],[A2901],[A2901],[],0,0,0,0,False,0.918001
43,A0301,62354,7839,1,1,0,0,[A0302],[A0302],[],[],0,0,0,0,False,0.914665
56,B4001,8466,14654,1,0,0,0,[B4103],[],[],[],0,0,0,0,False,0.955587
65,B1402,662,6500,2,2,2,0,"[B1401, B1403]","[B1401, B1403]","[B1401, B1403]",[],0,0,0,0,False,0.888218
70,B2705,21114,7455,9,6,3,0,"[B2704, B2710, B2709, B2706, B2703, B2708, B27...","[B2704, B2710, B2709, B2703, B2702, B2701]","[B2710, B2709, B2703]",[],0,0,0,0,False,0.896230
88,B1801,14436,9175,1,1,1,0,[B1803],[B1803],[B1803],[],0,0,0,0,False,0.929066
98,B3701,636,6890,0,0,0,0,[],[],[],[],0,0,0,0,False,0.816038


In [99]:
val_df

,epitope,label,MHC_expanded,peptide_length,key_binder,hla_sequence,predict_on,logit_1,logit_2,final_predictions,Length,predict_on_len,isotonic_prob,isotonic_pred
0,LSTVGPRL,1,C0304,8.0,537649.0,MRVMAPRTLILLLSGALALTETWAGSHSMRYFYTAVSRPGRGEPHF...,MRVMAPRTLILLLSGALALTETWAGSHSMRYFYTAVSRPGRGEPHF...,-1.635385,1.839033,0.862834,NaN,NaN,0.767125,1
1,AETTTLFQF,1,B4403,9.0,317702.0,MRVTAPRTLLLLLWGAVALTETWAGSHSMRYFYTAMSRPGRGEPRF...,MRVTAPRTLLLLLWGAVALTETWAGSHSMRYFYTAMSRPGRGEPRF...,-3.080864,3.342883,0.965871,NaN,NaN,0.995639,1
2,LPMKVRALGL,1,B0702,10.0,518233.0,MLVMAPRTVLLLLSAALALTETWAGSHSMRYFYTSVSRPGRGEPRF...,MLVMAPRTVLLLLSAALALTETWAGSHSMRYFYTSVSRPGRGEPRF...,-2.684186,2.932315,0.949421,NaN,NaN,0.981215,1
3,DIQSSGRAK,1,A0301,9.0,12400.0,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,3.406410,-3.716567,0.023740,NaN,NaN,0.006106,0
4,TMGHHTVGLK,1,A0301,10.0,308039.0,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,1.666246,-1.937590,0.125913,NaN,NaN,0.197259,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
787639,YILPILGICNE,0,C1701,NaN,NaN,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,2.986101,-3.262673,0.036874,11.0,383.0,0.019663,0
787640,SAPEEGEVSDS,0,C1701,NaN,NaN,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,3.538209,-3.794415,0.022001,11.0,383.0,0.007954,0
787641,GLAIVTKYITK,0,C1701,NaN,NaN,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,2.715949,-2.995777,0.047617,11.0,383.0,0.037881,0
787642,SKKVDFELAVQ,0,C1701,NaN,NaN,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,MRVMAPQALLLLLSGALALIETWAGSHSMRYFYTAVSRPGRGEPRF...,3.267602,-3.562536,0.027584,11.0,383.0,0.011199,0


In [107]:
train_df.query('((Allele == "A0201") & (label == 1))').groupby("Length").count()

,Allele,Peptide,label,allele_sequence,predict_on,predict_on_len,training,logit_1,logit_2,isotonic,isotonic_prob,isotonic_pred
Length,,,,,,,,,,,,
7,7,7,7,7,7,7,7,7,7,7,7,7
8,21,21,21,21,21,21,21,21,21,21,21,21
9,1508,1508,1508,1508,1508,1508,1508,1508,1508,1508,1508,1508
10,379,379,379,379,379,379,379,379,379,379,379,379
11,356,356,356,356,356,356,356,356,356,356,356,356
12,76,76,76,76,76,76,76,76,76,76,76,76
13,37,37,37,37,37,37,37,37,37,37,37,37
14,11,11,11,11,11,11,11,11,11,11,11,11
15,5,5,5,5,5,5,5,5,5,5,5,5


In [108]:
train_df.query('((Allele == "A0201") & (label == 0))').groupby("Length").count()

,Allele,Peptide,label,allele_sequence,predict_on,predict_on_len,training,logit_1,logit_2,isotonic,isotonic_prob,isotonic_pred
Length,,,,,,,,,,,,
7,31,31,31,31,31,31,31,31,31,31,31,31
8,110,110,110,110,110,110,110,110,110,110,110,110
9,7793,7793,7793,7793,7793,7793,7793,7793,7793,7793,7793,7793
10,1724,1724,1724,1724,1724,1724,1724,1724,1724,1724,1724,1724
11,1847,1847,1847,1847,1847,1847,1847,1847,1847,1847,1847,1847
12,383,383,383,383,383,383,383,383,383,383,383,383
13,181,181,181,181,181,181,181,181,181,181,181,181
14,57,57,57,57,57,57,57,57,57,57,57,57
15,32,32,32,32,32,32,32,32,32,32,32,32
